In [17]:
pip install pandas

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [18]:

pip install spotipy

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [19]:
%pip install urllib3 --upgrade 
%pip install requests --upgrade 
%pip install spotipy --upgrade

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [20]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import os
import numpy as np
import timeit
import pandas as pd
from operator import itemgetter
import time
import datetime as dt

os.environ['SPOTIPY_CLIENT_ID'] = '3191e7e8b04e46c1af64f49bcdd020be'
os.environ['SPOTIPY_CLIENT_SECRET'] = '88dc925eff0d4006bb6d0b9b2401ac59'

In [21]:
auth_manager = SpotifyClientCredentials()
sp = spotipy.Spotify(auth_manager=auth_manager)

In [22]:
import json
with open('mpd_slice_0_999.json') as json_file:
    data = json.load(json_file)

In [23]:
# calculate artist concentration value for a playlist
# low value means high concentration - maybe rename to artist diversity?
# input: 2D array of artists for each song
# output: real value between 0 and 1
def get_artist_conc(artists):
    unique_artists = len(set(artists))
    total_appearances = len(artists)
    artist_conc = unique_artists / total_appearances
    return artist_conc

In [24]:
MS_IN_DAY = 86400
def get_artists_and_dates(uris):
    # do the batching above this- here we assume that there are no more than 50 URIs
    # i could put an assert statement here but it would slow the code down
    # assert(len(uris) <= 50)
    artists = []
    dates = []
    tracks = sp.tracks(uris)['tracks']
    for track in tracks:
        if track is None:
            print(track)
        else:
            for artist in track['artists']:
                artists.append(artist['name'])
            date = track['album']['release_date']
            while len(date) < 10:
                date += '-01'
            ms_date = dt.datetime.timestamp(dt.datetime.strptime(date,"%Y-%m-%d"))
            dates.append(ms_date / MS_IN_DAY)
            
    return artists, dates

In [25]:
# get all audio features for a given playlist
def get_playlist_audio_features_batch(tracks):
    tracks = pd.DataFrame(tracks)
    playlist_uris = tracks['track_uri']

    remainder = len(tracks) % 100
    all_features = []
    if remainder > 0: 
        all_features = sp.audio_features(playlist_uris[:remainder])
    while remainder < len(tracks):
        all_features += sp.audio_features(playlist_uris[remainder:remainder+100])
        remainder += 100
    
    remainder = len(tracks) % 50
    artists, dates = [], []
    if remainder > 0:
        artists, dates = get_artists_and_dates(playlist_uris[:remainder])
    while remainder < len(tracks):
        next_artists, next_dates = get_artists_and_dates(playlist_uris[remainder:remainder+50])
        artists += next_artists
        dates += next_dates
        remainder += 50
    return all_features, artists, dates

In [26]:
# calculate aggregate audio features for a playlist
# currently only focused on audio features, I'll deal with artist and release date later
def get_playlist_aggregate_features_v2(tracks):
    
    all_features, artists, dates = get_playlist_audio_features_batch(tracks)
    all_features = pd.DataFrame(all_features)

    feature_list = ['danceability', 'energy', 'loudness', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo']
    aggregate_features = {}
    for feature in feature_list:
        all_by_feature = all_features[feature]
        aggregate_features[feature] = np.mean(all_by_feature)
        aggregate_features[feature+'_var'] = np.var(all_by_feature)
    aggregate_features['date'] = np.mean(dates)
    aggregate_features['date_var'] = np.var(dates)
    aggregate_features['artist_conc'] = get_artist_conc(artists)
    return aggregate_features

In [27]:
# build the aggregate dataset!
def build_dataset_v2(playlists):
    aggregate_dataset = []
    for playlist in playlists:
        aggregate_dataset.append(get_playlist_aggregate_features_v2(playlist['tracks']))
    return aggregate_dataset

In [28]:
dataset = pd.DataFrame(build_dataset_v2(data['playlists'][:10]))
dataset.to_csv(r'csvdata_10.csv')

In [30]:
print(dataset[3:6])

   danceability  danceability_var    energy  energy_var  loudness  \
3      0.514429          0.030802  0.620902    0.058971 -9.618754   
4      0.576235          0.028260  0.650418    0.040555 -7.634529   
5      0.683662          0.021363  0.717413    0.026690 -5.883962   

   loudness_var  speechiness  speechiness_var  acousticness  acousticness_var  \
3     18.742903     0.067000         0.004607      0.273514          0.103539   
4      6.233682     0.041218         0.000193      0.177189          0.054154   
5      5.831106     0.076234         0.004057      0.115931          0.035315   

   ...  instrumentalness_var  liveness  liveness_var   valence  valence_var  \
3  ...              0.096092  0.188278      0.019453  0.451258     0.061171   
4  ...              0.046579  0.166524      0.028347  0.490294     0.065056   
5  ...              0.000007  0.211870      0.027743  0.567538     0.035745   

        tempo   tempo_var          date      date_var  artist_conc  
3  125.54762